# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
np.array(y)[:5]


array([0, 0, 0, 1, 0])

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes2 = 3

# Define the number of neurons in the output layer
output_nodes = 1

In [ ]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_nodes, activation="sigmoid"))


In [ ]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 4s 7ms/step - loss: 0.7648 - accuracy: 0.5438
Epoch 2/50
38/38 [==============================] - 0s 8ms/step - loss: 0.6960 - accuracy: 0.5163
Epoch 3/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6814 - accuracy: 0.5146
Epoch 4/50
38/38 [==============================] - 1s 16ms/step - loss: 0.6735 - accuracy: 0.6197
Epoch 5/50
38/38 [==============================] - 0s 9ms/step - loss: 0.6713 - accuracy: 0.6230
Epoch 6/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6705 - accuracy: 0.6188
Epoch 7/50
38/38 [==============================] - 0s 9ms/step - loss: 0.6685 - accuracy: 0.6264
Epoch 8/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6649 - accuracy: 0.6222
Epoch 9/50
38/38 [==============================] - 0s 8ms/step - loss: 0.6633 - accuracy: 0.6280
Epoch 10/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6617 - accuracy: 0.6322
Epoch 11/50
38/38 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.6489 - accuracy: 0.6200 - 207ms/epoch - 16ms/step
Loss: 0.648928165435791, Accuracy: 0.6200000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
keras_file = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(keras_file)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path


# Load the model to a new object
nn_model2 = tf.keras.models.load_model(keras_file)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
predictions = nn_model2.predict(X_test,verbose=2)

# Display a sample of the predictions
predictions[:10]

13/13 - 0s - 112ms/epoch - 9ms/step


array([[0.4783745 ],
       [0.62230325],
       [0.59993196],
       [0.62230325],
       [0.5812277 ],
       [0.16699941],
       [0.25649133],
       [0.57431805],
       [0.5739491 ],
       [0.6213017 ]], dtype=float32)

In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_rounded = pd.DataFrame(predictions).round()

# Display a sample of the predictions
predictions_rounded.head(10)


,0
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
5,0.0
6,0.0
7,1.0
8,1.0
9,1.0


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_rounded))

              precision    recall  f1-score   support

           0       0.67      0.40      0.50       191
           1       0.60      0.82      0.69       209

    accuracy                           0.62       400
   macro avg       0.64      0.61      0.60       400
weighted avg       0.63      0.62      0.60       400



---
## Discuss creating a recommendation system for student loans

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

In order to recommend student loan options appropriate to a particular student we'll need information about loan options available.  We'll also need information about the student's financial needs with respect to their education.

**Loan programs to choose from** - A good recommendation system should have information about all the loan programs that the student could use, including the federal government's student loan programs, and offerings from other financial institutions, like Sallie Mae, and about the many loan forgiveness options available to some graduates.  The information must include:

1. Interest rates which will vary with loan terms.  A lower rate is more affordable.
2. Maximum and minimum loan amounts, to ensure the student's need for funds can be met.
3. Restrictions, such as minimum credit score, which can affect a student's eligibility
4. Financial institution ratings and reviews, to highlight banks that are good to work with in contrast to lower-rated institutions that provide poorer service.

**The student's needs and qualifications for a loan** - This information can be collected from the student applicant:
1. Field of study - some programs offer loan forgiveness to graduates
2. Co-signer information, because a co-signer can make the approval process quicker and make an application eligible for better rates based on co-signer financials.
3. Citizenship, which can effecting loan eligibility.
4. Grades earned, and level of education.
4. Financial picture including money already borrowed and income (if any).  For private loans, a credit score is needed.

This combination of available loan information and student application information will make the site better at matching loans to applicants.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Let's look at each filtering method.
- Collaborative filtering makes predictions on the history of similar users.  A student is grouped with similar students and the model will predict which loan options a given applicant will prefer.
- Content-based filtering looks at a student's past history to make predictions about the future.  This will not be so useful in this context because most students have little or no borrowing history from which to learn.
- Context-based filtering is like content-based filtering with a lot of additional attributes added - many of which will have nothing to do with money or loan payments.

In short, collaborative filtering is the way to go.  Each student applicant is clustered with students who had similar situations when they were shopping for student loans.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Implementation and operational challenges:
- Privacy - the information collected is going to include personally identifiable information (PII) and must be treated with extreme care.  Solution here is to discuss the site data requirements with the legal team and make prudent choices about what data to collect given the risks.  
- Accuracy of loan information - Loan rates change frequently, so the system will need a reliable way to load the rates at least every morning.  Inaccurate rates will cast doubt on the accuracy of other information in our application.
- Dependencies on third party sites - One helpful feature of our system would be links to the loan application page of financial institutions.  These will need to be checked frequently to avoid leaving broken links on the page, which would make our app seem unprofessional and unmaintained.

##References

Student loan forgiveness - https://www.salliemae.com/blog-new/student-loan-forgiveness-faq/

Student loan information - https://www.nerdwallet.com/h/category/student-loans




